In [1]:
from keras.utils import load_img, img_to_array

target_image_path='img/kot.jpeg'
style_reference_image_path='img/star.jpeg'

width, height =load_img(target_image_path).size

img_height=400
img_width=int(width*img_height/height)

In [2]:
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img=load_img(image_path,target_size=(img_height, img_width))
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x[:,:,0]+=103.939
    x[:,:,1]+=116.779
    x[:,:,2]+=123.68
    x=x[:,:,::-1]
    x=np.clip(x,0,255).astype('unit8')
    return x


In [3]:
from keras import backend as K

target_image=K.constant(preprocess_image(target_image_path))
style_reference_image=K.constant(preprocess_image(style_reference_image_path))
combination_image=K.placeholder((1,img_height,img_width,3))

input_tensor=K.concatenate([target_image,
                            style_reference_image,
                            combination_image], axis=0)

model=vgg19.VGG19(input_tensor=input_tensor,
                  weights='imagenet',
                  include_top=False)
print('Model został załadowany')

80134624/80134624 [==============================] - 3s 0us/step
Model został załadowany


In [4]:
def content_loss(base, combination):
    return K.sum(K.square(combination-base))

In [5]:
def gram_matrix(x): 
    features = K.batch_flatten (K.permute_dimensions (x, (2, 0, 1))) 
    gram=K.dot (features. K.transpose(features)) 
    return gram

def style_loss(style, combination):
    S=gram_matrix(style)
    C=gram_matrix(combination)
    channels=3
    size= img_height* img_width
    return K.Sum(K.square (S-C)) / (4. * (channels ** 2) * (size ** 2))

In [6]:
def total_variation_loss (x):
    a = K.square(
         x[:, :img_height -1, img_width- 1, :] -x[:, 1:, :img_width -1, :]) 
    b = K.square( 
         x[:, :img_height - 1, img_width -1, :] -x[:, :img_height- 1, 1:, :]) 
    return K.sum(K.pow(a + b, 1.25))

In [9]:
outputs_dict=dict([(layer.name, layer.output) for layer in model.layers])
content_layer= "block5_conv2"
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
total_variation_weight = 1e-4 
style_weight=1. 
content_weight= 0.025

loss=K.variable(0.)

layer_features = outputs_dict[content_layer] 
target_image_features =layer_features [0, :,:,:] 
combination_features= layer_features [2,:,:,:]
loss +=content_weight* content_loss (target_image_features,  combination_features)

for layer_name in style_layers:
    layer_features= outputs_dict[layer_name]
    style_reference_features =layer_features [1,:, :,:]
    combination_features= layer_features [2,:, :,:]
    sl = style_loss (style_reference_features, combination_features)
    loss +=(style_weight / len(style_layers)) * sl
loss += total_variation_weight* total_variation_loss (combination_image)


RuntimeError: `variable += value` with `tf.Variable`s is not supported. Use `variable.assign_add(value)` to modify the variable, or `out = variable + value` if you need to get a new output Tensor.

In [ ]:
grads= K.gradients (loss, combination_image)[0]

fetch_loss_and_grads= K.function([combination_image], [loss, grads])

class Evaluator (object):
    def _init_(self): 
        self. loss_value= None 
        self.grads_values= None
    def loss (self, x):
        assert self. loss_value is None 
        x-x.reshape((1, img_height, img_width, 3))
        outs =fetch_loss_and_grads ([x])
        loss_value =outs[0]
        grad_values =outs[1]. flatten().astype('float64')
        self.loss_value= loss_value
        self.grad_values= grad_values 
        return self.loss_value
    
    def grads (self, x):
        assert self.loss_value is not None
        grad_values =np.copy(self.grad_values)
        self.loss_value = None 
        self.grad_values = None
        return grad_values

evaluator =Evaluator()

In [ ]:
from scipy.optimize import fmin_1_bfgs_b 
from scipy.misc import imsave
import time

result_prefix='style_transfer_result' 
iterations =20

x=preprocess_image (target_image_path)
x=x.flatten()

for i in range(iterations): 
    print('Początek iteracji numer', 1) 
    start_time=time.time()
    x, min_val, info =fmin_1_bfgs_b(evaluator. loss, x,
                                    fprime=evaluator.grads,
                                    maxfun=20)
    print('Aktualna wartość straty:', min_val)
    img =x.copy().reshape((img_height, img_width, 3))
    img= deprocess_image(img)
    fname =result_prefix+'_at_iteration_%d.png' %i
    imsave(fname, img)
    end_time= time.time()
    print('Obraz zapisany w pliku. fname')
    print('Iteracia numer %d została zakończona w czasie %ds' % (i, end_time-start_time))